In [31]:
import src.kgbench as kg
from src.rgcn_explainer_utils import *
import torch



In [32]:
import os

# Get the current directory
current_dir = os.getcwd()

# Get the parent directory
parent_dir = os.path.dirname(current_dir)

# Check if the current directory is already the parent directory
if current_dir != '/Users/macoftraopia/Documents/GitHub/RGCN-Explainer':
    # Set the parent directory as the current directory
    os.chdir(parent_dir)
!pwd

/Users/macoftraopia/Documents/GitHub/RGCN-Explainer


In [47]:
import kgbench as kg
from src.rgcn_explainer_utils import *
name = 'dbo_gender'
if name in ['aifb', 'mutag', 'bgs', 'am', 'mdgenre']:
    data = kg.load(name, torch=True, final=False)
if 'IMDb' in name:    
    data = torch.load(f'data/IMDB/finals/{name}.pt')
if 'dbo' in name:
    data = torch.load(f'data/DBO/finals/{name}.pt')
get_relations(data)
relations = [data.i2rel[i][0] for i in range(len(data.i2rel))]
data.triples = torch.Tensor(data.triples).to(int)
data.withheld = torch.Tensor(data.withheld).to(int)
data.training = torch.Tensor(data.training).to(int)
print('rel:', data.num_relations, 'ent:', data.num_entities, 'triples:', data.triples.shape)
print('training', data.training.shape, 'withheld', data.withheld.shape)

rel: 225 ent: 11534 triples: torch.Size([15029, 3])
training torch.Size([466, 2]) withheld torch.Size([251, 2])


In [35]:
data.triples = torch.Tensor(data.triples).to(int)
data.withheld = torch.Tensor(data.withheld).to(int)
data.training = torch.Tensor(data.training).to(int)
data.entities = np.append(data.triples[:,0].detach().numpy(),(data.triples[:,2].detach().numpy()))
relations = get_relations(data)

#d_classes(data)

In [14]:
# val = data.withheld[:,0]
# most_common_node = {}
# for i in val:
#     sub_edges, neighborhoods, sub_edges_tensor = find_n_hop_neighbors(data, 2,int(i), adj=True)

#     counter = Counter([num for tup in sub_edges for num in tup]).most_common(1)[0][0]
#     most_common_node[int(i)] = counter

In [36]:
def frequency_relations(data):
    freq = Counter(data.triples[:,1].tolist())
    sorted_freq = {data.i2r[k]: v for k, v in sorted(freq.items(), key=lambda item: item[1], reverse=True)}
    return sorted_freq
f = frequency_relations(data)
print(f)

{'http://purl.org/dc/terms/subject': 3374, 'http://dbpedia.org/ontology/careerStation': 1930, 'http://dbpedia.org/ontology/team': 1442, 'http://dbpedia.org/ontology/birthPlace': 1332, 'http://www.w3.org/ns/prov#wasDerivedFrom': 984, 'http://xmlns.com/foaf/0.1/isPrimaryTopicOf': 948, 'http://purl.org/linguistics/gold/hypernym': 849, 'http://dbpedia.org/ontology/wikiPageExternalLink': 639, 'http://dbpedia.org/ontology/position': 313, 'http://dbpedia.org/ontology/occupation': 306, 'http://dbpedia.org/ontology/thumbnail': 274, 'http://dbpedia.org/ontology/deathPlace': 193, 'http://dbpedia.org/ontology/nationality': 171, 'http://dbpedia.org/ontology/managerClub': 157, 'http://xmlns.com/foaf/0.1/depiction': 146, 'http://xmlns.com/foaf/0.1/homepage': 131, 'http://dbpedia.org/ontology/genre': 93, 'http://dbpedia.org/ontology/almaMater': 87, 'http://dbpedia.org/ontology/formerTeam': 71, 'http://dbpedia.org/property/sport': 59, 'http://dbpedia.org/ontology/termPeriod': 58, 'http://dbpedia.org/on

In [38]:
import torch
import statistics
def select_matching_triples(triples_tensor, nodes_to_check):
    selected_triples = []

    # Iterate over the triples and check if any node matches with nodes_to_check
    for triple in triples_tensor:
        head, relation, tail = triple.tolist()
        if head in nodes_to_check or tail in nodes_to_check:
            selected_triples.append(triple)

    return torch.stack(selected_triples)
val = data.withheld[:,0]

selected_triples = select_matching_triples(data.triples, val)

def compute_average_node_degree(triples_tensor):
    # Create an empty dictionary to store the node degrees
    node_degrees = {}

    # Iterate over the triples and calculate the node degrees
    for triple in triples_tensor:
        head, relation, tail = triple.tolist()
        # Increment the degree for the head node
        node_degrees[head] = node_degrees.get(head, 0) + 1
        # Increment the degree for the tail node
        node_degrees[tail] = node_degrees.get(tail, 0) + 1

    # Compute the average node degree
    num_nodes = len(node_degrees)
    total_degree = sum(node_degrees.values())
    average_degree = total_degree / num_nodes
        # Compute the standard deviation of node degrees
    degree_values = list(node_degrees.values())
    degree_std = statistics.stdev(degree_values)

    return average_degree, degree_std


# Example usage
triples_tensor = torch.tensor([[12264, 20, 19861],
                               [22808, 0, 22261],
                               [21753, 9, 13458]])

average_degree, std_degree = compute_average_node_degree(data.triples)
print("Average Node Degree:", np.round(average_degree,3), "±", np.round(std_degree,3))


Average Node Degree: 2.606 ± 6.427


In [39]:
import torch
import statistics

def compute_average_degree_at_2_hops(triples_tensor):
    # Create an adjacency dictionary to store the neighbors of each node
    adjacency_dict = {}

    # Iterate over the triples and build the adjacency dictionary
    for triple in triples_tensor:
        head, relation, tail = triple.tolist()

        # Add tail as a neighbor of head
        if head not in adjacency_dict:
            adjacency_dict[head] = set()
        adjacency_dict[head].add(tail)

        # Add head as a neighbor of tail
        if tail not in adjacency_dict:
            adjacency_dict[tail] = set()
        adjacency_dict[tail].add(head)

    # Compute the average degree at 2 hops for each node
    average_degrees_2_hops = []
    for node, neighbors in adjacency_dict.items():
        two_hop_neighbors = set()
        for neighbor in neighbors:
            two_hop_neighbors.update(adjacency_dict.get(neighbor, set()))
        average_degrees_2_hops.append(len(neighbors) + len(two_hop_neighbors))

    # Compute the average degree at 2 hops and its standard deviation
    average_degree_2_hops = statistics.mean(average_degrees_2_hops)
    degree_std_2_hops = statistics.stdev(average_degrees_2_hops)

    return average_degree_2_hops, degree_std_2_hops


# Example usage
triples_tensor = torch.tensor([[1, 20, 2],
                               [2, 0, 3],
                               [3, 9, 4],
                               [4, 7, 1]])

average_degree_2_hops, degree_std_2_hops = compute_average_degree_at_2_hops(data.triples)
print("Average Degree at 2 Hops:", np.round(average_degree_2_hops,3),'±', np.round(degree_std_2_hops,3))



Average Degree at 2 Hops: 43.298 ± 93.297


In [40]:
import statistics
def select_matching_triples(triples_tensor, nodes_to_check):
    selected_triples = []

    # Iterate over the triples and check if any node matches with nodes_to_check
    for triple in triples_tensor:
        head, relation, tail = triple.tolist()
        if head in nodes_to_check or tail in nodes_to_check:
            selected_triples.append(triple)

    return torch.stack(selected_triples)
val = data.withheld[:,0]

selected_triples = select_matching_triples(data.triples, val)

def compute_num_edges_to_node(triples_tensor, node):
    # Create an adjacency dictionary to store the neighbors of each node
    adjacency_dict = {}

    # Iterate over the triples and build the adjacency dictionary
    for triple in triples_tensor:
        head, relation, tail = triple.tolist()

        # Add tail as a neighbor of head
        if head not in adjacency_dict:
            adjacency_dict[head] = set()
        adjacency_dict[head].add(tail)

        # Add head as a neighbor of tail
        if tail not in adjacency_dict:
            adjacency_dict[tail] = set()
        adjacency_dict[tail].add(head)

    # Compute the number of edges to the given node at 0 and 2 hops
    num_edges_to_node_0_hop = len(adjacency_dict.get(node, set()))
    neighbors_2_hops = set()
    if node in adjacency_dict:
        neighbors = adjacency_dict[node]
        for neighbor in neighbors:
            if neighbor in adjacency_dict:
                neighbors_2_hops.update(adjacency_dict[neighbor])
    num_edges_to_node_2_hops = len(neighbors_2_hops)

    return num_edges_to_node_0_hop, num_edges_to_node_2_hops

sum_0,sum_2 = [], []
for i in val:
    #print(int(i))
    num_edges_0_hop, num_edges_2_hops = compute_num_edges_to_node(data.triples, int(i))
    sum_0.append(num_edges_0_hop)
    sum_2.append(num_edges_2_hops)
print('avg num edges 0 hop', np.round(np.mean(sum_0),3), '±', np.round(np.std(sum_0),3))
print('avg num edges 2 hop', np.round(np.mean(sum_2),3), '±', np.round(np.std(sum_2),3))
# Example usage




avg num edges 0 hop 16.394 ± 9.23
avg num edges 2 hop 213.793 ± 155.405


In [ ]:
def object_type(v,h,data, relation_id = None,type=True):
    ''' Get the object class of a specific relation'''
    if type:
        relation_id = [i for i in range(data.num_relations) if 'type' in data.i2r[i]][-1]
    output_indices_v, output_values, value_indices = select_relation(v, data.num_entities, relation_id)
    output_indices_h, output_values, value_indices = select_relation(h, data.num_entities, relation_id)
    objects_types = match_to_triples(output_indices_v, output_indices_h,data, sparse=False)
    list = []
    for i in objects_types:
        list.append(data.i2e[i[2]][0])#.split('#')[1])
    result = Counter(list)
    return result

for node_idx in data.withheld[:,0]:
    h, v = torch.load(f'chk/aifb_chk/hops_2_size_5e-05_lr_0.1_ent_-1_type_1_threshold_0.5_init_const_exp_/masked_adj/masked_ver{node_idx}'), torch.load(f'chk/aifb_chk/hops_2_size_5e-05_lr_0.1_ent_-1_type_1_threshold_0.5_init_const_exp_/masked_adj/masked_hor{node_idx}')
    print(f'node {node_idx}:', object_type(v,h,data, 39))